In [1]:
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
from time import time
from glob import glob
from scipy.io import loadmat
from PIL import Image
from scipy.stats import pearsonr
from hrf_fitting.src.feature_weighted_rf_models import make_rf_table,receptive_fields, model_space, prediction_menu,bigmult
from hrf_fitting.src.feature_weighted_rf_models import train_fwrf_model
from hrf_fitting.src.gabor_feature_dictionaries import gabor_feature_maps
from os.path import join

%load_ext autoreload
%autoreload 2
#%matplotlib inline

Using gpu device 0: GeForce GTX TITAN X


In [2]:
##known: number of voxels
nvox = 22701

#### load a set of model predictions from caffe

In [3]:
from_caffe = h5py.File('/media/tnaselar/Data/deepnet_vim-1/feature_weighted_models/VoxelPrediction_stimVal_imgIdx_3.hdf5', 'r')
print from_caffe.keys()

[u'prob', u'voxel1', u'voxel2', u'voxel3', u'voxel4', u'voxel5']


In [4]:
caffe_preds = np.zeros((1,nvox))
for k in from_caffe.keys()[1:]:
    caffe_preds += from_caffe[k]
    
    

In [5]:
#### read in deepnet model in native theano format

In [6]:
import pickle
version_number =  '0p2'
saving_place = '/media/tnaselar/Data/deepnet_vim-1/feature_weighted_models/'
saving_file = 'model_space_'+version_number+'.p'


In [7]:
join(saving_place, saving_file)

'/media/tnaselar/Data/deepnet_vim-1/feature_weighted_models/model_space_0p2.p'

In [8]:
ms = pickle.load(open(join(saving_place, saving_file), 'r'))

In [9]:
##known stimulus parameters
Ttrn = 1750
Tval = 120

##don't use these
junk_keys = ['__header__', '__globals__', '__version__', 'fc6', 'fc7', 'fc8','prob']



deepnet_val_feature_dict = loadmat('/media/tnaselar/Data/deepnet_vim-1/vim-1_val_response.mat')
##it contains some key/value pairs we don't want
deepnet_val_feature_dict = {key: value.astype('float32') for key, value in deepnet_val_feature_dict.items() if key not in junk_keys}
print deepnet_val_feature_dict.keys()

##construct model-space tensor
val_mst = ms.construct_model_space_tensor(deepnet_val_feature_dict)

['conv3', 'conv2', 'conv1', 'conv5', 'conv4']
-----------feature: conv3
at pixel resolution 13 the following rfs will default to 0: (array([ 0.75]),)
-----------feature: conv2
-----------feature: conv1
-----------feature: conv5
at pixel resolution 13 the following rfs will default to 0: (array([ 0.75]),)
-----------feature: conv4
at pixel resolution 13 the following rfs will default to 0: (array([ 0.75]),)
model_space_tensor has been z-scored


In [10]:

##generate predictions one voxel at a time
pred = np.zeros((Tval,nvox))
for v in range(nvox):  
    pred[:,v] = np.squeeze(bigmult(val_mst[np.newaxis,ms.optimal_rf_model[v],:,:],
                                   ms.optimal_feature_weights[np.newaxis,:,v, np.newaxis]))

In [11]:
pred.shape

(120, 22701)

In [12]:
plt.plot(pred[3,:], caffe_preds[0,:], '.')


In [13]:
plt.show()